In [4]:
# only for sw_emu and hw_emu
!emconfigutil -f xilinx_u280_xdma_201920_3 --nd 1


****** configutil v2021.2 (64-bit)
  **** SW Build 3363252 on 2021-10-14-04:41:01
    ** Copyright 1986-2020 Xilinx, Inc. All Rights Reserved.

INFO: [ConfigUtil 60-895]   Target platform: /opt/xilinx/platforms/xilinx_u280_xdma_201920_3/xilinx_u280_xdma_201920_3.xpfm
INFO: [ConfigUtil 60-1578]   This platform contains Xilinx Shell Archive '/opt/xilinx/platforms/xilinx_u280_xdma_201920_3/hw/xilinx_u280_xdma_201920_3.xsa'
INFO: [ConfigUtil 60-1032] Extracting hardware platform to ./.Xil/configutil-25825-caadlab-01/xilinx_u280_xdma_201920_3
emulation configuration file `emconfig.json` is created in current working directory 


In [5]:
import os
os.environ['EMCONFIG_PATH'] = os.environ['PWD']

In [6]:
os.environ['XCL_EMULATION_MODE'] = 'hw_emu'

In [7]:
import pynq
import numpy as np
from vnx_utils import *

In [8]:
for i in range(len(pynq.Device.devices)):
    print("{}) {}".format(i, pynq.Device.devices[i].name))

0) xilinx_u280_xdma_201920_3
1) xilinx_u280_xdma_201920_3


In [ ]:
currentDevice = pynq.Device.devices[0]
xclbin = '/home/pouya/github/G-FPin_HW_new/G-FPin_HW/Vitis/hw.xclbin'
ol = pynq.Overlay(xclbin,device=currentDevice)

In [ ]:
ol.ip_dict

In [ ]:
mm2s = ol.krnl_mm2s_0
s2mm = ol.krnl_s2mm_0
CGRA_krnl = ol.rtl_kernel_CGRA_accelerator_0

size = 1408 * 100
shape = (size,1)
num_col = 2
num_entry_config_table = 64
size_inst_buf = num_entry_config_table * num_col
shape_inst_buf = (size_inst_buf, 1)
size_HBM = 512 # ignore this b/c it is host-independent (controlled by the FPGA)
shape_HBM = (size_HBM, 1)

if hasattr(ol, 'HBM0'):
    mm2s_buf = pynq.allocate(shape, dtype=np.uint8, target=ol.HBM0)
    s2mm_buf = pynq.allocate(shape, dtype=np.uint8, target=ol.HBM0)
    CGRA_inst_buf = pynq.allocate(shape_inst_buf, dtype=np.uint32, target=ol.HBM0)
    CGRA_HBM_buf1 = pynq.allocate(shape_HBM, dtype=np.uint8, target=ol.HBM0)
    CGRA_HBM_buf2 = pynq.allocate(shape_HBM, dtype=np.uint8, target=ol.HBM0)
    
else:
    mm2s_buf = pynq.allocate(shape, dtype=np.uint8, target=ol.bank1)
    s2mm_buf = pynq.allocate(shape, dtype=np.uint8, target=ol.bank1)
    CGRA_inst_buf = pynq.allocate(shape_inst_buf, dtype=np.uint32, target=ol.bank1)
    CGRA_HBM_buf1 = pynq.allocate(shape_HBM, dtype=np.uint8, target=ol.bank1)
    CGRA_HBM_buf2 = pynq.allocate(shape_HBM, dtype=np.uint8, target=ol.bank1)

mm2s_buf[:] = np.random.randint(low=0, high=((2**8)-1), size=shape, dtype=np.uint8)
CGRA_inst_buf[:] = np.random.randint(low=0, high=((2**8)-1), size=shape_inst_buf, dtype=np.uint8)
CGRA_HBM_buf1[:] = np.random.randint(low=0, high=((2**8)-1), size=shape_HBM, dtype=np.uint8)
CGRA_HBM_buf2[:] = np.random.randint(low=0, high=((2**8)-1), size=shape_HBM, dtype=np.uint8)

In [ ]:
mm2s_buf.sync_to_device()
CGRA_inst_buf.sync_to_device()
CGRA_HBM_buf1.sync_to_device() # we might even no need that
CGRA_HBM_buf2.sync_to_device() # we might even no need that

In [ ]:
mm2s.start(mm2s_buf,size, 1) # Async
CGRA_krnl.start(CGRA_inst_buf, CGRA_HBM_buf1, CGRA_HBM_buf2) # Async
s2mm.call(s2mm_buf,size) # Sync

In [ ]:
s2mm_buf.sync_from_device()

In [ ]:
msg = "SUCCESS!" if np.array_equal(mm2s_buf, s2mm_buf) else "FAILURE!"
print(msg)

In [ ]:
del mm2s_buf
del s2mm_buf
del CGRA_inst_buf
del CGRA_HBM_buf1
del CGRA_HBM_buf2
pynq.Overlay.free(ol)